In [1]:
# %pip install -q sentencepiece
# %pip install -q protobuf
# %pip install -q -U bitsandbytes
# %pip install -q -U git+https://github.com/huggingface/transformers.git
# %pip install -q -U git+https://github.com/huggingface/peft.git
# %pip install -q -U git+https://github.com/huggingface/accelerate.git
# %pip install -q scipy
# %pip install -q -U trl

In [2]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel, PeftConfig, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import Dataset, concatenate_datasets
import pandas as pd
import math

/home/user/mambaforge/envs/tensorml/lib/python3.11/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [3]:
from torch import nn
import re
import ast
import numpy as np

class CustomSFTTrainer(SFTTrainer):
    def __init__(self, tokenizer, *args, **kwargs):
        super(CustomSFTTrainer, self).__init__(*args, **kwargs)
        self.tokenizer = tokenizer

    def compute_loss(self, model, inputs, return_outputs=False):
        inputs_copy = inputs.copy()
        labels = inputs_copy.pop("labels")
        outputs = model(**inputs_copy)
        logits = outputs.logits
        
        # Compute the loss using the superclass's compute_loss method
        super_loss, _ = super().compute_loss(model, inputs, return_outputs=True)
        print("\npreloss:", super_loss.item())
        loss = super_loss
        
        wrong_proportion = 1
        
        labels_decoded = decoded = self.tokenizer.batch_decode(labels.tolist())
        labels_result = labels_decoded[0]
        
        try:
            target_match = re.search(r"Test_1_Output=\[\[.*?\]\]", labels_result)
            target_array_string = target_match.group(0).replace("Test_1_Output=", "")
            target_array = np.array(ast.literal_eval(target_array_string))
            # Apply softmax to logits to get probabilities
            probs = torch.softmax(logits, dim=-1)
            # Get the predicted tokens (highest probability)
            _, predictions = torch.max(probs, dim=-1)
            decoded = self.tokenizer.batch_decode(predictions.tolist())
            predictions_result = decoded[0]
            match1 = re.search(r"Test_1_Output=\[\[.*?\]\]", predictions_result)
            match2 = re.search(r"Hello_1_Output=\[\[.*?\]\]", predictions_result)
            match3 = re.search(r"To_1_Output=\[\[.*?\]\]", predictions_result)
            predicted_array_string = ""
            if match1:
                predicted_array_string = match1.group(0).replace("Test_1_Output=", "")
                loss *= 0.5
                print("match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            elif match2:
                predicted_array_string = match2.group(0).replace("Hello_1_Output=", "")
                loss *= 0.95
                print("match2")
            elif match3:
                predicted_array_string = match3.group(0).replace("To_1_Output=", "")
                loss *= 0.95
                print("match3")

            if predicted_array_string != "":
                print("predicted_array_string != empty string")
                try:
                    predicted_array = np.array(ast.literal_eval(predicted_array_string))
                    if predicted_array.shape == target_array.shape:
                        print("correct shape")
                        loss *= 0.5
                        equal_elements_count = np.sum(np.equal(target_array, predicted_array))
                        total_elements_count = target_array.size
                        wrong_elements_count = total_elements_count - equal_elements_count
                        wrong_proportion = wrong_elements_count / total_elements_count
                        loss *= wrong_proportion
                except:
                    pass
        except:
            pass
        
        print("wp:", wrong_proportion, "  loss:", loss.item())

        return (loss, outputs) if return_outputs else loss

In [4]:
def generate_prompt(data_point):
    user_message = data_point["instruction"]
    assistant_message = data_point["output"]
    text = f"<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant\n{assistant_message}<|im_end|>"
    return text

In [5]:
base_model_name = "microsoft/Orca-2-13b"
new_adapter_name = "Orca-2-13b-4bit-finetune-on-ARC-11"

# Load the entire model on the GPU 0
device_map = {"": 0}

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

In [6]:
# Load the data using pandas
data_file = "data/ARC_augmented_training_puzzles.json"
df = pd.read_json(data_file)
# Convert the pandas dataframe to a dataset
dataset1 = Dataset.from_pandas(df)

# Load the data using pandas
data_file = "data/ARC_augmented_evaluation_puzzles.json"
df = pd.read_json(data_file)
# Convert the pandas dataframe to a dataset
dataset2 = Dataset.from_pandas(df)

# Load the data using pandas
data_file = "data/core_ARC_puzzles.json"
df = pd.read_json(data_file)
# Convert the pandas dataframe to a dataset
dataset3 = Dataset.from_pandas(df)

# Generate the "prompt" column for each dataset
dataset1 = dataset1.map(lambda data_point: {"prompt": generate_prompt(data_point)})
dataset2 = dataset2.map(lambda data_point: {"prompt": generate_prompt(data_point)})
dataset3 = dataset3.map(lambda data_point: {"prompt": generate_prompt(data_point)})

# # Assuming dataset1 and dataset2 are instances of Dataset from the `datasets` library
dataset = concatenate_datasets([dataset1, dataset2, dataset3])

Map:   0%|          | 0/11240 [00:00<?, ? examples/s]

Map:   0%|          | 0/72504 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [7]:
def main():
    # Load base model
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        quantization_config=bnb_config,
        device_map=device_map,
    )
    model.config.use_cache = False
    model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model)  

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model_name, add_eos_token=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training

    per_device_train_batch_size = 1
    gradient_accumulation_steps = 1
    max_seq_length = 5000

    output_dir = "results/" + new_adapter_name

    steps_per_epoch = len(dataset) // (
        per_device_train_batch_size * gradient_accumulation_steps
    )
    print("Steps:", steps_per_epoch)

    # Set training parameters
    training_arguments = TrainingArguments(
        output_dir=output_dir,
        max_steps=steps_per_epoch,
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        optim="paged_adamw_8bit",
        lr_scheduler_type="cosine",
        save_strategy="steps",
        evaluation_strategy="no",
        save_steps=100,
        logging_steps=1,
        learning_rate=5e-6,
        fp16=True,
        warmup_steps=0.03,
        group_by_length=True,
        gradient_checkpointing=True,
    )

    trainer = CustomSFTTrainer(
        model=model,
        train_dataset=dataset,
        peft_config=peft_config,
        dataset_text_field="prompt",
        max_seq_length=max_seq_length,
        tokenizer=tokenizer,
        args=training_arguments,
    )

    # Train model
    trainer.train()

    # Save trained model
    adapter = "adapters/" + new_adapter_name
    trainer.model.save_pretrained(adapter)

In [ ]:
main()

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Steps: 93744


Map:   0%|          | 0/93744 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/user/mambaforge/envs/tensorml/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(



preloss: 0.7598862051963806
match2
predicted_array_string != empty string
wp: 1   loss: 0.7218918800354004


...


preloss: 0.1749018430709839
match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
predicted_array_string != empty string
correct shape
wp: 0.03571428571428571   loss: 0.0015616236487403512

preloss: 0.09043259173631668
wp: 1   loss: 0.09043259173631668

preloss: 0.3964417576789856
match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
predicted_array_string != empty string
correct shape
wp: 0.01384083044982699   loss: 0.0013717708643525839

preloss: 0.3845233619213104
match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
predicted_array_string != empty string
correct shape
wp: 0.0875   loss: 0.008411448448896408

preloss: 0.08181115984916687
match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
predicted_array_string != empty string
wp: 1   loss: 0.040905579924583435

preloss: 0.20360004901885986
match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
predicted_array_string != empty string
correct shape
wp: 0.013333333333333334   loss: 0.0006786668673157692

preloss: 0.18927842378616333
match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



preloss: 0.09432181715965271
match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
predicted_array_string != empty string
wp: 1   loss: 0.047160908579826355

preloss: 0.07676130533218384
wp: 1   loss: 0.07676130533218384

preloss: 0.11298234015703201
match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
predicted_array_string != empty string
correct shape
wp: 0.08   loss: 0.002259646775200963

preloss: 0.27085527777671814
match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
predicted_array_string != empty string
correct shape
wp: 0.04861111111111111   loss: 0.0032916441559791565

preloss: 0.47634953260421753
wp: 1   loss: 0.47634953260421753

preloss: 0.23518870770931244
match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
predicted_array_string != empty string
correct shape
wp: 0.0234375   loss: 0.0013780588051304221

preloss: 0.06432850658893585
match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
predicted_array_string != empty string
correct shape
wp: 0.05   loss: 0.0008041063556447625

preloss: 0.11321992427110672
match1

/home/user/mambaforge/envs/tensorml/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(



preloss: 0.05378086119890213
wp: 1   loss: 0.05378086119890213

preloss: 0.2155909240245819
wp: 1   loss: 0.2155909240245819

preloss: 0.06087905913591385
match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
predicted_array_string != empty string
correct shape
wp: 0.03787878787878788   loss: 0.0005765062524005771

preloss: 0.2808622419834137
match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
predicted_array_string != empty string
correct shape
wp: 0.03333333333333333   loss: 0.002340518869459629

preloss: 0.1660986840724945
match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
predicted_array_string != empty string
wp: 1   loss: 0.08304934203624725

preloss: 0.11061878502368927
match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
predicted_array_string != empty string
correct shape
wp: 0.18   loss: 0.004977845586836338

preloss: 1.1336896419525146
match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
predicted_array_string != empty string
wp: 1   loss: 0.5668448209762573

preloss: 0.09348923712968826
wp: 1   loss: 0.0934892


preloss: 0.6268996596336365
match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
predicted_array_string != empty string
correct shape
wp: 0.047619047619047616   loss: 0.007463091518729925

preloss: 0.3645557463169098
wp: 1   loss: 0.3645557463169098

preloss: 0.13485532999038696
match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
predicted_array_string != empty string
correct shape
wp: 0.04844290657439446   loss: 0.0016331960214301944

preloss: 0.17270854115486145
wp: 1   loss: 0.17270854115486145

preloss: 1.132941484451294
match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
predicted_array_string != empty string
wp: 1   loss: 0.566470742225647

preloss: 0.07404154539108276
match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
predicted_array_string != empty string
correct shape
wp: 0.015625   loss: 0.00028922478668391705

preloss: 0.058178164064884186
match1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
predicted_array_string != empty string
correct shape
wp: 0.016666666666666666   loss: 0.00024240903439931571

preloss: 